In [6]:
import tensorflow as tf
import numpy as np
import pandas as pd

# Import Stock Dataset

In [8]:
# APPLE stock data from 2006-07-18 to 2020-07-18
data = pd.read_csv('./AAPL.csv')
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2006-07-17,7.390000,7.587143,7.378572,7.481429,6.476653,256135600
1,2006-07-18,7.594285,7.692857,7.407143,7.557143,6.542199,250112100
2,2006-07-19,7.565714,7.868571,7.480000,7.728571,6.690604,347685800
3,2006-07-20,8.708571,8.798572,8.531428,8.642858,7.482098,493036600
4,2006-07-21,8.545714,8.735714,8.520000,8.674286,7.509306,222973100
